In [9]:
import numpy as np
import pandas as pd
import requests

from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans


In [5]:
# Import Data
print('Load Description Embeddings')
desc_emb = np.load('../data/desc_emb.npz')
desc_embeddings = dict(zip(("desc_embeddings"), (desc_emb[k] for k in desc_emb)))['d']
print(desc_embeddings.shape)

print('Load Description Labels, Address Labels and Name Labels')
NFT_attributes = np.load('../data/NFT_attributes.npz')
d = dict(zip(("description_list","contract_addresses","name_list"), (NFT_attributes[k] for k in NFT_attributes)))
description_list = d['description_list']
contract_addresses = d['contract_addresses']
name_list = d['name_list']
print(description_list.shape)

Load Description Embeddings
(11083, 768)
Load Description Labels, Address Labels and Name Labels
(11083,)


In [42]:
# Get Owner Adress + NFTs
OWNER = '0x2dCaf47e340b3a5C0f1779111697C39696E3Dc21'

URL = f'https://eth-mainnet.alchemyapi.io/v2/demo/getNFTs/?owner={OWNER}'
Owner_NFTs = requests.get(URL).json()['ownedNfts']

print(Owner_NFTs[0]['contract']['address'])
contract_addresses = []
token_ids = []
for nft in Owner_NFTs:
    contract_addresses.append(nft['contract']['address'])
    token_ids.append(nft['id']['tokenId'])



0x60b74dea92950b9f6564b2c5a7d20482f20f6dc1


In [40]:
# Get NFT Description
Owner_descriptions = []
Owner_available_addresses = []
Owner_names = []

for address in contract_addresses:
    URL = f"https://eth-mainnet.alchemyapi.io/v2/demo/getNFTMetadata?contractAddress={address}&tokenId=1&tokenType=erc721"
    NFT = requests.get(URL)
    try:
        NFTjson = NFT.json()
    except:
        continue
    if (type(NFTjson) is dict and "metadata" in NFTjson.keys() and type(NFTjson["metadata"]) is dict and "description" in NFTjson["metadata"].keys()):
        print('This is what we want')
        external_url = NFTjson["metadata"]["description"]
        Owner_descriptions.append(external_url)
        Owner_available_addresses.append(address)
        name = "Not Available"
        if (type(NFTjson["metadata"]) is dict and "title" in NFTjson["metadata"].keys()):
            name = NFTjson["metadata"]["title"]
        Owner_names.append(name)

print(Owner_descriptions[0])

This is what we want
This is what we want
Mint here https://ecotars.art Ecotars 🤖 a 3D animated NFT collection that cares about the environment ♻️🌎 Founder & creative director: @RKropachek🧑‍💻 


In [ ]:
print('Loading Model')
model = SentenceTransformer('bert-base-nli-mean-tokens')

#description embeddings
print(f'Calculating embeddings for {len(Owner_descriptions)} descriptions')
Owner_desc_embeddings = model.encode(Owner_descriptions, normalize_embeddings=True, show_progress_bar=True, batch_size = 64)